# Net present value and profitability

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlalchemy as sq
import geopandas as gp
import census
import os
from shapely.geometry import Polygon
from glob import glob
import matplotlib.patches as mpatch
import scipy.optimize
from pandas.tseries.offsets import MonthEnd

# NB remove when estimating discount rate
discount_rate_ex = 0.06813142841167415

capi = census.Census(os.environ['CENSUS_API_KEY'])

In [ ]:
plt.style.use('asu-light')

In [ ]:
def weighted_percentile (vals, percentiles, weights):
    if len(vals) != len(weights):
        raise ArgumentError('values and weights arrays are not same length!')

    nas = pd.isnull(vals) | pd.isnull(weights)

    nnas = np.sum(nas)
    if nnas > 0:
        warn(f'found {nnas} NAs in data, dropping them')

    vals = vals[~nas]
    weights = weights[~nas]

    weights = weights / np.sum(weights)
    sortIdx = np.argsort(vals)
    vals = vals.iloc[sortIdx]
    weights = weights.iloc[sortIdx]

    cumWeights = np.cumsum(weights)
    if not isinstance(percentiles, np.ndarray):
        percentiles = np.array(percentiles)
    percentiles = percentiles / 100

    # center weights, i.e. put the point value halfway through the weight
    # https://github.com/nudomarinero/wquantiles/blob/master/wquantiles.py
    centeredCumWeights = cumWeights - 0.5 * weights
    return np.interp(percentiles, centeredCumWeights, vals)

In [ ]:
new = None
ex = None

In [ ]:
DISCOUNT_RATE_NEW = 0.08 #0.065
DISCOUNT_RATE_EX = 0.04125 #1.04 / 1.014 - 1
#DISCOUNT_RATE_EX = DISCOUNT_RATE_NEW
CAP_RATE = 0.04424375
APPRECIATION = 0.014 # 0.014
SCALE_FACTOR = 1.8031293436149882  # Scale factor to bring IPUMS rents in line with Zillow rental index
VACANCY_RATE = 0.04  # 4% assumed vacancy
TRANSACTION_COSTS = 0.09

DB_URI = 'postgresql://matthewc@localhost:5432/matthewc'


In [ ]:
DISCOUNT_RATE_EX

In [ ]:
if new is None:
    new = pd.read_sql('''
        SELECT c.gid, c.clean_apn AS apn, c.county, c.puma, prototype, c.total_cost, c.total_rent, fit_sfh_duplex, fit_fourplex AS fit_threeplex, fit_sixplex
            FROM diss.building_costs c
            LEFT JOIN diss.building_fit f ON (c.gid = f.gid)
            LEFT JOIN diss.gp16 p ON (p.gid = c.gid)
            WHERE (p.total_rent IS NOT NULL OR p.vacant_npv IS NOT NULL) -- make sure existing rent was estimated
            AND p.puma IS NOT NULL
    ''', DB_URI)

In [ ]:
if ex is None:
    ex = pd.read_sql('''
    WITH most_recent_sales AS (
        SELECT "Main_SalesPriceAmount" AS price, "Main_RecordingDate" AS sale_date, "PropertyInfo_ImportParcelID" AS importparcelid,
            ROW_NUMBER() OVER (PARTITION BY "PropertyInfo_ImportParcelID" ORDER BY "Main_RecordingDate" DESC) AS sale_number
        FROM diss.ztrans t
        WHERE SUBSTRING(t."Main_RecordingDate", 1, 4) IN ('2013', '2014', '2015', '2016', '2017')
    )
    SELECT county, p.clean_apn AS apn,
        p.gid, puma, tract, hqta, total_rent, vacant_npv, lu16, price, sale_date
        FROM diss.gp16 p
        LEFT JOIN most_recent_sales s ON (p.Main_ImportParcelId = s.importparcelid AND s.sale_number = 1)
        WHERE p.scag_zn_co IN (
            '1110', -- single family residential
            '1111', -- high dens SF residential
            '1112', -- med dens SF residential
            '1113', -- low dens SF residential
            '1150'  -- rural residential
        )
        AND p.Building_PropertyLandUseStndCode IN ('RR101', 'VL101')
        AND (total_rent IS NOT NULL OR vacant_npv IS NOT NULL)
        AND p.puma IS NOT NULL
        ''', DB_URI)

In [ ]:
len(ex)

In [ ]:
len(new)

In [ ]:
assert (len(new) / len(ex)) == 4

In [ ]:
(ex.total_rent * SCALE_FACTOR).describe().round()

In [ ]:
def compute_results (ex, new, discount_rate_ex, discount_rate_new, appreciation, cap_rate, op_cost, cost_scale, name):
    if new is not None:
        # see short-circuit code path used in calibration, where new is None
        new['noi'] = new.total_rent * (1 - op_cost) * 12 * SCALE_FACTOR * (1 - VACANCY_RATE)  # annualize
    ex['noi'] = ex.total_rent * (1 - op_cost) * 12 * SCALE_FACTOR * (1 - VACANCY_RATE)
    
    # since noi does not change year to year, p-v is just a multiplier of noi
    ex_value = ex.vacant_npv.fillna(
        ex.noi * sum([1 / ((1 + discount_rate_ex)**i * (1 + appreciation) ** i) for i in range(10)])  # rental value
        + ex.noi / cap_rate / (1 + discount_rate_ex) ** 10 * (1 + appreciation) ** 10 * (1 - TRANSACTION_COSTS) # ultimate sale value
    )
    
    if new is None:
        # short-circuit code path used in calibration
        return ex_value
    else:
        ex['value'] = ex_value
        
    new['value'] = (
        new.noi * sum([1 / ((1 + discount_rate_new)**i ) for i in range(2, 10)])  # rental value
        + new.noi / cap_rate / (1 + discount_rate_new) ** 10 * (1 + appreciation) ** 10 * (1 - TRANSACTION_COSTS)  # ultimate sale value
        - new.total_cost * cost_scale / 2  # Construction cost, year one
        - new.total_cost * cost_scale / 2 / (1 + discount_rate_new)  # Construction cost, year 2
    )
    
    print(ex.value.describe().round())
    median_ex = np.percentile(ex.value, 50)
    
    # account for the fit
    new.loc[new.prototype.isin(['sfh', 'duplex']) & ~new.fit_sfh_duplex, 'value'] = -np.inf
    new.loc[new.prototype.isin(['threeplex']) & ~new.fit_threeplex, 'value'] = -np.inf
    new.loc[new.prototype.isin(['sixplex']) & ~new.fit_sixplex, 'value'] = -np.inf
        
    values = (v := new.set_index(['gid', 'prototype'])).value.unstack()
    values.head().round()
    
    values = (
        values.merge(ex[['gid', 'value', 'hqta']]
                     .rename(columns={'value': 'existing'}), left_index=True, right_on='gid', how='left', validate='1:1')
                    .set_index('gid')
    )
    assert not values.existing.isnull().any()
    
    values.to_parquet(f'../data/{name}_net_present_value.parquet')
    
    hqta_values = values.copy()
    hqta_values.loc[~hqta_values.hqta, ['sfh', 'duplex', 'threeplex', 'sixplex']] = -np.inf
    hqta_values.to_parquet(f'../data/{name}_hqta_net_present_value.parquet')
    
    most_profitable = (
        values[['existing', 'sfh', 'duplex', 'threeplex', 'sixplex']]
        .apply(lambda r: r.idxmax(), 1).astype('category').cat.set_categories(['existing', 'sfh', 'duplex', 'threeplex', 'sixplex'])
    )
    
    counts = pd.DataFrame({'Total': most_profitable.value_counts(), 'Percent': (most_profitable.value_counts(normalize=True) * 100).round(1).astype('str') + '%'}).transpose()[['existing', 'duplex', 'threeplex', 'sixplex']]\
        .rename(columns={'existing': 'No change', 'sfh': 'New single-family', 'threeplex': 'Threeplex', 'duplex': 'Duplex', 'sixplex': 'Sixplex'})
    total_parcels = counts.loc['Total', :].sum()
    counts['Total'] = 'test'
    counts.loc['Total', 'Total'] = total_parcels
    counts.loc['Percent', 'Total'] = '100.0%'
    
    ex = ex.set_index('gid')
    ex['most_profitable'] = most_profitable.reindex(ex.index)    
    ex['not_existing'] = ex.most_profitable != 'existing'
    locations = ex.groupby('tract').not_existing.mean() * 100
    
    v = most_profitable.value_counts()
    total = v.duplex + v.threeplex * 2 + v.sixplex * 5 # duplex has one marginal unit etc
    # add vacant lots
    total += (~ex.vacant_npv.isnull() & ex.not_existing).sum()  # 1 add'l unit for vacant lots
    print(f'total units: {total}')
    
    display(counts)
    
    hqta_profitable = ex.loc[ex.hqta, 'most_profitable']
    hqta_counts = pd.DataFrame({'Total': hqta_profitable.value_counts(), 'Percent': (hqta_profitable.value_counts(normalize=True) * 100).round(1).astype('str') + '%'}).transpose()[['existing', 'duplex', 'threeplex', 'sixplex']]\
        .rename(columns={'existing': 'No change', 'threeplex': 'Threeplex', 'duplex': 'Duplex', 'sixplex': 'Sixplex'})
    total_units = hqta_counts.loc['Total', :].sum()
    hqta_counts['Total'] = 'test'
    hqta_counts.loc['Total', 'Total'] = total_units
    hqta_counts.loc['Percent', 'Total'] = '100.0%'
    v = hqta_profitable.value_counts()
    hqta_total = v.duplex + v.threeplex * 2 + v.sixplex * 5 # duplex has one marginal unit etc
    # add vacant lots
    hqta_total += (ex.not_existing & ex.hqta & ~ex.vacant_npv.isnull()).sum()  # 1 addl unit for vacant lots
    
    return counts, hqta_counts, locations, total, hqta_total, median_ex

In [ ]:
scenarios = {
    'Current appreciation':  {
        'discount_rate_new': 0.11,
        'discount_rate_ex': 0.06,
        'cap_rate': CAP_RATE,
        'appreciation': 0.0480328437009123,
        'op_cost': 0.45,
        'cost_scale': 1.1
    }
}

## Calibrate the single-family discount rate

The cap rate and appreciation rate are estimated from data, while the discount rate is just made up, and a wide range of values are plausible. Calibrate the discount rate so that the median home is estimated exactly correctly based on past sale values - i.e. 50% of homes have their values underestimated, and 50% have them overestimated.

In [ ]:
ex_with_sales = ex.loc[~ex.price.isnull() & (ex.price > 5000)].copy()

In [ ]:
# Put all sale values in 2017 dollars
# apply inflation using Zillow Home Value Index for LA, SFR only
# https://www.zillow.com/research/data/
zhvi = pd.read_csv('../data/Metro_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_mon.csv')
zhvi_la = zhvi.loc[zhvi.RegionName == 'Los Angeles-Long Beach-Anaheim, CA', [i for i in zhvi.columns if i.startswith('20')]].iloc[0]
# scale to appropriate scale factors
zhvi_la /= zhvi_la.loc['2017-12-31'] # this is the end of the ZTrans data and also the end of the PUMS data used to estimate rents

In [ ]:
ex_with_sales['date'] = pd.to_datetime(ex_with_sales.sale_date)

# MonthEnd(0) - end of this month, even if this is the end of the month.
ex_with_sales['sale_month'] = (ex_with_sales.date + MonthEnd(0)).astype(str)

ex_with_sales.sale_month.value_counts().sort_index()

In [ ]:
ex_with_sales['zhvi'] = zhvi_la.loc[ex_with_sales.sale_month].values
assert not ex_with_sales.zhvi.isnull().any()
ex_with_sales['sale_price17'] = ex_with_sales.price / ex_with_sales.zhvi

In [ ]:
def log_median_npv_to_sale_ratio (discount_rate):
    scenario = {**scenarios['Current appreciation'], 'discount_rate_ex': discount_rate}
    npvs = compute_results(ex_with_sales, None, name=None, **scenario).reindex(ex_with_sales.index)
    return np.log(np.percentile(npvs / ex_with_sales.sale_price17, 50)) # log makes root at zero, and also (I think) linearizes somehow?

In [ ]:
discount_root = scipy.optimize.root_scalar(log_median_npv_to_sale_ratio, x0=0.05, x1=0.06, method='secant', xtol=1e-5)
discount_rate_ex = discount_root.root
assert discount_root.converged
discount_root

In [ ]:
# Update tge current appreciation scenario with the discount rate we found
scenarios['Current appreciation']['discount_rate_ex'] = discount_rate_ex

In [ ]:
ex_with_sales['current_npv'] = current_npvs = compute_results(ex_with_sales, None, name=None, **scenarios['Current appreciation'])

In [ ]:
r2_data = ex_with_sales[ex_with_sales.sale_price17 <= 2e6]

In [ ]:
r2 = 1 - np.sum((r2_data.current_npv - r2_data.sale_price17) ** 2) / np.sum((r2_data.sale_price17 - r2_data.sale_price17.mean()) ** 2)
r2

In [ ]:
plt.subplots(figsize=(9, 6))
plt.scatter(ex_with_sales.sale_price17, ex_with_sales.current_npv, s=0.0001, color='black')
plt.plot([0, 5e6], [0, 5e6], color='C5', lw=2)
plt.plot([0, 5e6], [0, 5e6], color='C0', lw=0.75)
plt.xlim(0, 2e6)
plt.ylim(0, 1.5e6)
plt.xticks([0, 5e5, 1e6, 1.5e6, 2e6], ['$0', '$500k', '$1m', '$1.5m', '$2m'])
plt.yticks([0, 5e5, 1e6, 1.5e6], ['$0', '$500k', '$1m', '$1.5m'])
plt.xlabel('Sale price (2017 dollars)')
plt.ylabel('Predicted net present value')
plt.savefig('../../dissertation/fig/construction/npv_vs_sale.png', bbox_inches='tight', dpi=300)

## Run for multiple scenarios

In [ ]:
scenarios['Base'] = {
    'discount_rate_new': 0.11,
    'discount_rate_ex': discount_rate_ex,
    'cap_rate': CAP_RATE,
    'appreciation': 0.014,  # CPI
    'op_cost': 0.45,
    'cost_scale': 1.1 # 10% contingency
}

scenarios['Low discount rate'] = {
    'discount_rate_new': 0.08,
    'discount_rate_ex': 0.04125,
    'cap_rate': CAP_RATE,
    'appreciation': 0.014,  # CPI
    'op_cost': 0.45,
    'cost_scale': 1.1 # 10% contingency
}

scenarios['Equal discount rate (8% existing and new)'] = {
    'discount_rate_new': 0.08,
    'discount_rate_ex': 0.08,
    'cap_rate': CAP_RATE,
    'appreciation': 0.014,  # CPI
    'op_cost': 0.45,
    'cost_scale': 1.1 # 10% contingency
}

scenarios['Low operating cost (25%)'] = {
    'discount_rate_new': 0.11,
    'discount_rate_ex': discount_rate_ex,
    'cap_rate': CAP_RATE,
    'appreciation': 0.0480328437009123,
    'op_cost': 0.25,
    'cost_scale': 1.1 # 10% contingency
}

scenarios['High construction cost'] = {
    'discount_rate_new': 0.11,
    'discount_rate_ex': discount_rate_ex,
    'cap_rate': CAP_RATE,
    'appreciation': 0.014,  # CPI
    'op_cost': 0.45,
    'cost_scale': 1.4 # 30% on top
}

In [ ]:
out = {}
for name, srio in scenarios.items():
    print(name)
    out[name] = dict(zip(('counts', 'hqta_counts', 'locations', 'total_units',  'hqta_total_counts', 'median_val_ex'), compute_results(ex.copy(), new.copy(), name=name, **srio)))

In [ ]:
# dump scenarios to a handy-dandy table
scenario_table = (
    (pd.DataFrame(scenarios).transpose() * 100).apply(lambda c: c.round(3).astype('str') + '%')
        .rename(columns={
            'cap_rate': 'Capitalization rate',
            'appreciation': 'Appreciation rate',
            'op_cost': 'Operating cost'
        }))
print(scenario_table.to_latex())

OLD CODE BELOW HERE.
## Sensitivity test table

In [ ]:
output = []

pct = '{:.2f}%'.format

for sname, scenario in scenarios.items():
    output.append(np.array([[f'\\centering\\textbf{{{sname}}}', '', '', '', '', '', '']]))
    output.append(np.array([
        [f'\\textit{{{i}}}' for i in
            ['Marginal units', 'Discount rate (new construction)', 'Discount rate (existing structure)', 'Capitalization rate', 'Appreciation rate', 'Operating cost', '']
        ],
        [
            '{:,d}'.format(out[sname]['total_units']),
            pct(scenario['discount_rate_new'] * 100),
            pct(scenario['discount_rate_ex'] * 100),
            pct(scenario['cap_rate'] * 100),
            pct(scenario['appreciation'] * 100),
            pct(scenario['op_cost']),
            ''
        ]
    ]))
    cts = out[sname]['counts'].copy()
    cts.loc['Total', :] = cts.loc['Total', :].apply('{:,d}'.format)
    cts = cts.reset_index()
    output.append(np.array([[f'\\textit{{{i}}}' if i != 'index' else '' for i in cts.columns]]))
    output.append(cts.to_numpy())

res = pd.DataFrame(np.concatenate(output))
res

In [ ]:
print(res.to_latex(escape=False, index=False))

In [ ]:
scenarios

## HQTA scenario

In [ ]:
hqta_profitable = ex.loc[ex.hqta, 'most_profitable']
counts = pd.DataFrame({'Total': hqta_profitable.value_counts(), 'Percent': (hqta_profitable.value_counts(normalize=True) * 100).round(1).astype('str') + '%'}).transpose()[['existing', 'duplex', 'threeplex', 'sixplex']]\
    .rename(columns={'existing': 'No change', 'threeplex': 'Threeplex', 'duplex': 'Duplex', 'sixplex': 'Sixplex'})
counts.insert(1, 'New single-family', [0, '0.0%'])
total_units = counts.loc['Total', :].sum()
counts['Total'] = 'test'
counts.loc['Total', 'Total'] = total_units
counts.loc['Percent', 'Total'] = '100.0%'
counts

In [ ]:
counts.Duplex + counts.Threeplex * 2 + counts.Sixplex * 5

In [ ]:
counts.Duplex + counts.Threeplex + counts.Sixplex

In [ ]:
print(counts.to_latex())

In [ ]:
219884 / 545130

In [ ]:
# https://censusreporter.org/profiles/05000US06037-los-angeles-county-ca/
EXISTING_UNITS = 3_579_423
219884 / EXISTING_UNITS

## Gentrification???

In [ ]:
pct_poverty = pd.DataFrame(capi.acs5.state_county_tract(['B17001_001E', 'B17001_002E', 'B25001_001E', 'B03002_001E', 'B03002_003E'], '06', '037', census.ALL, year=2017))

In [ ]:
pct_poverty['poverty_rate'] = pct_poverty.B17001_002E / pct_poverty.B17001_001E * 100
pct_poverty['pct_white'] = pct_poverty.B03002_003E / pct_poverty.B03002_001E * 100

In [ ]:
pct_poverty['GEOID'] = pct_poverty.state.str.cat(pct_poverty.county).str.cat(pct_poverty.tract)

In [ ]:
devpoverty = ex.merge(pct_poverty, left_on='tract', right_on='GEOID', how='left', validate='m:1')
#assert not devpoverty.poverty_rate.isnull().any()  # TODO missing 5 tracts
with pd.option_context('display.float_format', '{:.1f}%'.format):
    vals = pd.concat([
        devpoverty[devpoverty.not_existing][['poverty_rate', 'pct_white']].quantile([0.05, 0.25, 0.50, 0.75, 0.95]),
        devpoverty[['poverty_rate', 'pct_white']].quantile([0.05, 0.25, 0.50, 0.75, 0.95])
    ],
        keys=['Redeveloped parcels', 'All parcels'],
        axis=1
    )
    
    vals.index = ['5th percentile', '25th percentile', 'Median', '75th percentile', '95th percentile']
    
    display(vals)
    print(vals.to_latex())
        

### Compare to overall housing stock

In [ ]:
devpoverty[devpoverty.poverty_rate.isnull()].tract_x.unique()

In [ ]:
with pd.option_context('display.max_colwidth', None):
    display(pd.DataFrame(capi.acs5.tables()).set_index('name').loc['B03002'])

In [ ]:
pd.DataFrame(capi.acs5.tables()).set_index('name').loc['B02001']